In [13]:
%reload_ext autoreload
%autoreload 2

In [14]:
from result_saver import SaverProvider
provider = SaverProvider()

In [15]:
import numpy as np
from Scratch import metadata_loader

md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 10
md = md[md["distance"] == max_distance]
md = md.sort_values(by='backend_name', ascending=False)

md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
51,2023-10-29 14:43:41.752908+01:00,bigger_rep_codes,ibm_sherbrooke,cmz633cvayrg008sp2a0,[],10000.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,10.0,10,0,_is_hex=True,Run bigger Repetition codes
242,2023-10-30 09:43:20.984078+01:00,bigger_rep_codes,ibm_sherbrooke,cmzpsa5vayrg008cpk30,[],10000.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,10.0,10,0,_is_hex=True,Run bigger Repetition codes v4: new distances ...


In [16]:
memories = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()

memories

{'mmr_log_0': array([[-1.69043800e+07+1.52777920e+07j, -6.95536570e+07+3.97118220e+07j,
         -8.61951720e+07-6.66768820e+07j, ...,
         -1.11575833e+08-8.60894400e+07j,  3.03813270e+07+6.08026020e+07j,
         -4.81019930e+07-6.95120170e+07j],
        [-1.93997510e+07+1.77208380e+07j, -5.59600780e+07+6.58611030e+07j,
         -6.72833050e+07-6.30488450e+07j, ...,
          6.76670320e+07-9.64343640e+07j,  4.18878580e+07+3.72742840e+07j,
         -4.42934960e+07-5.69044300e+07j],
        [-2.02618610e+07+2.40441530e+07j, -5.47100890e+07+6.68740870e+07j,
         -7.64413350e+07-7.37752260e+07j, ...,
         -1.11808223e+08-9.38663590e+07j, -3.56569670e+07+6.30416770e+07j,
         -4.44256210e+07-5.80749620e+07j],
        ...,
        [-1.25824540e+07+1.02028210e+07j,  3.95282030e+07+7.85776400e+07j,
         -8.98716200e+07-6.66634720e+07j, ...,
          6.82267440e+07-9.14290790e+07j, -4.78136750e+07+7.53373550e+07j,
         -4.03697040e+07-3.61357170e+07j],
        [-1.14

In [17]:
import pickle
from soft_info import get_repcode_layout, get_KDEs, get_counts
from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit_qec.decoders import UnionFindDecoder

layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend("ibm_sherbrooke"), _is_hex=True)

kde_dict, scaler_dict = get_KDEs(provider, 'ibm_sherbrooke', layout, bandwidths=0.2, plot=False)
print(kde_dict)

code = RepetitionCodeCircuit(max_distance, max_distance)
#decoder = UnionFindDecoder(code)

with open(f'dist_{max_distance}_decoder.pkl', 'rb') as f:
    decoder = pickle.load(f)

{114: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 116: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 118: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 120: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 122: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 104: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 102: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 100: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 98: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 113: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 115: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 117: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 119: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 121: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)], 111: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.

In [18]:
from soft_info import get_counts

#counts = get_counts(memories['mmr_log_0'], kde_dict, scaler_dict, layout, max_distance, verbose=True)
graph = decoder.decoding_graph.graph

print(graph.node_indexes()[:2])
print(graph.nodes()[10])

[0, 1]
{'is_boundary': False, 'time': 1, 'qubits': [4, 5], 'index': 4, 'properties': {}}


In [19]:
from soft_info import soft_reweight, get_counts, rx_draw_2D
from tqdm import tqdm

VERBOSE = False

memory = memories['mmr_log_0']

logical_counts = {'0':0, '1':0}
for shot in tqdm(range(len(memory))[:]):
    
    with open(f"dist_{max_distance}_decoder.pkl", 'rb') as f: # Reload the decoder each time to reset the graph
        decoder = pickle.load(f)

    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    

    reweighted_graph = soft_reweight(decoder, IQ_data, kde_dict, scaler_dict, layout)
    decoder.decoding_graph.graph = reweighted_graph

    

    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    
    logical_counts[f"{logical[0]}"] += 1

    if logical[0] == 0:
        continue
    
    print(logical_counts)

    if VERBOSE:
        print("Decoding graph after reweighting:")
        rx_draw_2D(decoder.decoding_graph.graph, atypical_nodes=code.string2nodes(count_key))
    
    if VERBOSE:
        print("\nmeas_str:", count_key)
        print("flipped_qubit_dict", flipped_qubit_dict)
        print("logical:", logical, "\n")
        if logical[0] == 1:
            print("------------ Wrong logical! ------------\n")

    

print( "\nLogical_counts:", logical_counts)


  0%|          | 4/10000 [00:00<13:12, 12.61it/s]

{'0': 1, '1': 1}


  1%|          | 100/10000 [00:08<14:34, 11.32it/s]

{'0': 97, '1': 2}


  1%|          | 110/10000 [00:09<13:55, 11.83it/s]

...

{'0': 252, '1': 5}


  4%|▎         | 366/10000 [00:32<15:12, 10.55it/s]

{'0': 359, '1': 6}


  5%|▍         | 492/10000 [00:42<13:50, 11.44it/s]


KeyboardInterrupt: 

In [ ]:
from soft_info import soft_reweight, get_counts, rx_draw_2D
from tqdm import tqdm

VERBOSE = False

memory = memories['mmr_log_0']

logical_counts = {'0':0, '1':0}
for shot in tqdm(range(len(memory))[:]):
    
    with open(f"dist_{max_distance}_decoder.pkl", 'rb') as f: # Reload the decoder each time to reset the graph
        decoder = pickle.load(f)

    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    if VERBOSE:
        print("Decoding graph before reweighting:")
        rx_draw_2D(decoder.graph)

    #reweighted_graph = soft_reweight(decoder, IQ_data, kde_dict, scaler_dict, layout)
    #decoder.graph = reweighted_graph

    if VERBOSE:
        print("Decoding graph after reweighting:")
        rx_draw_2D(decoder.graph)

    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    
    if VERBOSE:
        print("\nmeas_str:", count_key)
        print("flipped_qubit_dict", flipped_qubit_dict)
        print("logical:", logical, "\n")
        if logical[0] == 1:
            print("------------ Wrong logical! ------------\n")

    logical_counts[f"{logical[0]}"] += 1

print( "\nLogical_counts:", logical_counts)


100%|██████████| 20/20 [00:15<00:00,  1.29it/s]


Logical_counts: {'0': 20, '1': 0}
